In [1]:
%pip install mne
%pip install matplotlib
%pip install pymatreader
%pip install scipy


Note: you may need to restart the kernel to use updated packages.
^C
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install FOOOF

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install neurodsp

Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import matplotlib
import mne
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
# the following import is required for matplotlib < 3.2:
from mpl_toolkits.mplot3d import Axes3D  # noqa

from neurodsp.spectral import compute_spectrum, trim_spectrum


# Import NeuroDSP plotting functions
from neurodsp.plts import (plot_time_series, plot_power_spectra, plot_bursts, plot_lagged_coherence)

from fooof import FOOOF
import os
import pandas as pd
import csv

import scipy
from scipy import stats
from collections import Counter

# **The Following functions are functions for pre-processing the data**

In [ ]:
%cd workingDir

In [7]:
def segment_channels(raw): ## segments the data into desired channels
  channels_to_keep = ['E1', 'E2', 'E3', 'E4', 'E8', 'E9', 'E10', 'E15', 'E16', 'E18', 'E22', 'E23', 'E24', 'E25', 'E26', 'E27', 'E32', 'E33', 'E122', 'E123', 'E124']
  return raw.pick_channels(channels_to_keep)

In [8]:
def avgChannels(raw): ## averages channels into one evoked object of the data for the frontal lobe
  events = mne.find_events(raw, stim_channel = None)
  epochs = mne.Epochs(raw, events, event_id=None, tmin=0, tmax=10)
  evoked = epochs.average()
  return evoked

In [9]:
def computeFOOOF(raw):  ## FOOOF computations
  data, times = raw[:,:]
  freqs, powers = compute_spectrum(data[0], fs, method='welch', avg_type='median')
  freqs, powers = trim_spectrum(freqs, powers, [3, 28])
  return fm.report(freqs, powers, [3,28])

# **These blocks create necessary global variables for preprocessing, set our working directory, creating the FOOOF report, renaming files, and creating the PANDAS dataframe**

In [10]:

groups=dict(frontal = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])
fs = 250

In [11]:
#PANDAS Lists to be appended then made into Data frame
PersonalID = []
age = []
sex =[]

Aper_offset = []
Aper_exp = []
variance = []
error = []

ProcessedDataDict = {'ID': PersonalID, 'Age': age, 'Sex': sex,
'Aper_offset': Aper_offset, 'Aper_exp': Aper_exp, 'Variance': variance, 'Error': error}

PeakID = []
PeakIndex = []
CenterFrequency = []
PeakWidth = []
BandWidth = []

ProcessedPeaksDataDict = {'ID': PeakID, 'Age': age, 'Sex': sex, 'Peak_Number': PeakIndex , 'CF': CenterFrequency, 'PW': PeakWidth, 'BW': BandWidth}

failedIDs = []
redundantIDs = []
otherError = []

In [7]:
%cd workingDir

C:\Users\ishaa\Documents\PolygenceResearchProject\code\EEGPipelineTesterBatch


Completed so don't run once renamed

In [16]:

# for filename in os.listdir():
#   if filename.endswith('.fdt'):
#     os.rename(filename, filename[::-1][22::][::-1] + '.fdt')  #renames the files
#   elif filename.endswith('.set'):
#     os.rename(filename, filename[::-1][22::][::-1] + '.set')
# print('successfully completed')

# **Loop for preprocessing the files, creating FOOOF report, and saving the report to pandas.**

In [18]:
phenoDF = pd.read_csv('CorrectPhenotypesRaw.csv')  #phenotpypic data frame
phenoDF = phenoDF.set_index('EID')

fm = FOOOF()

Cell runs the analysis and saves values to be made into data frame

In [ ]:
for IDs, row in phenoDF.iterrows():
  if IDs not in PersonalID:
    try:
      raw = mne.io.read_raw_eeglab(input_fname = r'EEGFileDir' + '\sub-' + IDs + '.set') #read the EEG file that was renamed earlier in format ID.set
      raw = segment_channels(raw)
      raw = mne.channels.combine_channels(raw, groups, method = 'median')  #channel averaging
      report = computeFOOOF(raw) #FOOOF object to break apart and save in pandas df



      ## Saving results to Lists to be made into data frame ##

      #from CSV

      ProcessedDataDict['ID'].append(IDs)
      ProcessedDataDict['Age'].append(phenoDF.loc[IDs,'Age'])
      ProcessedDataDict['Sex'].append(phenoDF.loc[IDs,'Sex'])
      
      #from FOOOF
      ProcessedDataDict['Aper_offset'].append(fm.aperiodic_params_[0])
      ProcessedDataDict['Aper_exp'].append(fm.aperiodic_params_[-1])
      ProcessedDataDict['Variance'].append(fm.r_squared_)
      ProcessedDataDict['Error'].append(fm.error_)

      PeakNum = 0 
      for peak in fm.peak_params_:  #appends the IDs and peak parameters into the lists
        ProcessedPeaksDataDict['ID'].append(IDs)
        ProcessedPeaksDataDict['Peak_Number'].append(PeakNum)
        ProcessedPeaksDataDict['CF'].append(peak[0])
        ProcessedPeaksDataDict['PW'].append(peak[1])
        ProcessedPeaksDataDict['BW'].append(peak[2])
        PeakNum +=1
    except FileNotFoundError:
      failedIDs.append(IDs)
      continue
    except Exception as e:
      errorLog = (IDs, e)
      otherError.append(errorLog)
  else:
    redundantIDs.append(IDs)



In [ ]:
ProcessedPeaksDataDict['Age'] = []
ProcessedPeaksDataDict['Sex'] = []
for uniqueID in ProcessedPeaksDataDict['ID']:
    ProcessedPeaksDataDict['Age'].append(phenoDF.loc[uniqueID,'Age'])
    ProcessedPeaksDataDict['Sex'].append(phenoDF.loc[uniqueID,'Sex'])
print(len(ProcessedPeaksDataDict['Age']))
print(len(ProcessedPeaksDataDict['Sex']))

In [30]:
processedDF = pd.DataFrame(ProcessedDataDict)
processedDF.to_csv('ProcessedAperData.csv', index = False)

In [44]:

#multilevel data frame for peaks
ProcessedPeaksDF = pd.DataFrame(ProcessedPeaksDataDict)
ProcessedPeaksDF.sort_values(['ID', 'Peak_Number']).set_index(['ID', 'Peak_Number'])
ProcessedPeaksDF.to_csv('ProcessedPerData.csv', index = False)

# **The Following Section is to Run Null Hyphothesis Testing**

In [5]:
%cd workingDir

C:\Users\ishaa\Documents\PolygenceResearchProject\code\EEGPipelineTesterBatch


In [6]:
aperDF = pd.read_csv('ProcessedAperData.csv')
aperDF = aperDF.set_index('ID')

perDF = pd.read_csv('ProcessedPerData.csv')
perDF = perDF.set_index('ID')

## *Data Frame for test results*

In [7]:
Test = [] #format 'independent var name v dependent var name'
TestSex = [] #The associated sex for the test (each test will have both a M and F test)
PearsonR_correlationCoefficient = []
PearsonR_pValue = []

chiSquared_degreesOfFreedom = []
chiSquared_SampleSize = []
chiSquared_correlationCoefficient = []
chiSquared_pValue = []



index_values = pd.MultiIndex.from_arrays([Test, TestSex], names=['Test', 'Sex'])


df_pearson = pd.DataFrame(data={
    'r': PearsonR_correlationCoefficient,
    'Pearson P': PearsonR_pValue
})


df_chi_squared = pd.DataFrame(data={
    'df': chiSquared_degreesOfFreedom,
    'N': chiSquared_SampleSize,
    'X²': chiSquared_correlationCoefficient,
    'Chi-squared P': chiSquared_pValue
})

# Combine the DataFrames into the MultiIndex DataFrame
NullHypothesisTestingDF = pd.concat([df_pearson, df_chi_squared], axis=1,
                                    keys=['Pearson Correlation', 'Chi-squared'])

# Assign the MultiIndex to the DataFrame
NullHypothesisTestingDF.index = index_values


print(NullHypothesisTestingDF)



Empty DataFrame
Columns: [(Pearson Correlation, r), (Pearson Correlation, Pearson P), (Chi-squared, df), (Chi-squared, N), (Chi-squared, X²), (Chi-squared, Chi-squared P)]
Index: []


## *The following Chi - Squared tests treat age in groups 5 - 8 (research is 5-8 so groups of age 5 are lumped into this 6-8), 9 - 12, and 13 - 22 (research is 13 - late 20s and oldest subject is 22), rounding age to the nearest year and seperating the tests for male v female then testing the dependent variable against sex to confirm prior research. The continuous test uses a pearson test to check for correlation for the null hypothesis, treating age as continuous instead of groups like the chi - Squared tests* 

 *Test for Age V. Offset Female & Male      |      (H0) Offset is independent of Age      |      (H1) Offset varies based on an individual's Age*

In [8]:
#Run for 0s (Women) and 1s (Male)   to get the Age and Aperiodic Offset sorted by Men and Women
DependentVariable_Women = []
IndependentVariable_Women = []

DependentVariable_Men = []
IndependentVariable_Men = []

for ID, row in aperDF.iterrows():
    if aperDF.loc[ID, 'Sex'] == 0.0:
        DependentVariable_Women.append(aperDF.loc[ID, 'Age'])
        IndependentVariable_Women.append(aperDF.loc[ID, 'Aper_offset'])
    elif aperDF.loc[ID, 'Sex'] == 1.0:
        DependentVariable_Men.append(aperDF.loc[ID, 'Age'])
        IndependentVariable_Men.append(aperDF.loc[ID, 'Aper_offset'])
    else:
        raise

    if len(DependentVariable_Women) != len(IndependentVariable_Women):   #Just to be extra safe in case a loc raises an error if you use this code for a different use case
        raise Exception('Check again, the variables are not the same length')

    if len(DependentVariable_Men) != len(IndependentVariable_Men):       #Just to be extra safe in case a loc raises an error if you use this code for a different use case
        raise Exception('Check again, the variables are not the same length')

In [9]:
#sort the data into groups for both men and women (see above group explanation) for the chi squared test 

youngestOffsets_Women = []
middleOffsets_Women = []
highestOffsets_Women = []

youngestOffsets_Men = []
middleOffsets_Men = []
highestOffsets_Men = []
for ID, row in aperDF.iterrows():
    if aperDF.loc[ID, 'Sex'] == 0.0:
        if (5 <= round(aperDF.loc[ID, 'Age']) <= 8):
            youngestOffsets_Women.append(round(aperDF.loc[ID, 'Aper_offset'], 2))
        elif (9 <= round(aperDF.loc[ID, 'Age']) <= 12):
            middleOffsets_Women.append(round(aperDF.loc[ID, 'Aper_offset'], 2))
        elif (13 <= round(aperDF.loc[ID, 'Age']) <= 22):
            highestOffsets_Women.append(round(aperDF.loc[ID, 'Aper_offset'], 2))
        else:   
            raise Exception('Participant Age is outside of expected range')
    elif aperDF.loc[ID, 'Sex'] == 1.0:
        if (5 <= round(aperDF.loc[ID, 'Age']) <= 8):
            youngestOffsets_Men.append(round(aperDF.loc[ID, 'Aper_offset'], 2))
        elif (9 <= round(aperDF.loc[ID, 'Age']) <= 12):
            middleOffsets_Men.append(round(aperDF.loc[ID, 'Aper_offset'], 2))
        elif (13 <= round(aperDF.loc[ID, 'Age']) <= 22):
            highestOffsets_Men.append(round(aperDF.loc[ID, 'Aper_offset'], 2))
        else:   
            raise Exception('Participant Age is outside of expected range')



In [10]:
#Count the values and create contingency table for women

youngestFrequencies_Women = Counter(youngestOffsets_Women)
middleFrequencies_Women = Counter(middleOffsets_Women)
highestFrequencies_Women = Counter(highestOffsets_Women)

chiYoungestDF_Women = pd.DataFrame.from_dict(youngestFrequencies_Women, orient='index', columns=['6 - 8'])
chiMiddleDF_Women = pd.DataFrame.from_dict(middleFrequencies_Women, orient='index', columns=['9 - 12 '])
chiHighestDF_Women = pd.DataFrame.from_dict(highestFrequencies_Women, orient='index', columns=['13 - 22'])

chiSquared_DF_Women = chiMiddleDF_Women.combine_first(chiHighestDF_Women.combine_first(chiYoungestDF_Women))
chiSquared_DF_Women = chiSquared_DF_Women.fillna(0)
print(chiSquared_DF_Women)

        13 - 22  6 - 8  9 - 12 
-14.42      0.0    1.0      0.0
-13.21      0.0    0.0      1.0
-13.11      0.0    1.0      0.0
-13.10      1.0    0.0      0.0
-13.09      0.0    1.0      0.0
...         ...    ...      ...
-8.24       0.0    0.0      1.0
-8.13       0.0    1.0      0.0
-7.67       1.0    0.0      0.0
-7.56       0.0    1.0      0.0
-6.97       0.0    1.0      0.0

[307 rows x 3 columns]


In [11]:
#Count the values and create contingency table for men

youngestFrequencies_Men = Counter(youngestOffsets_Men)
middleFrequencies_Men = Counter(middleOffsets_Men)
highestFrequencies_Men = Counter(highestOffsets_Men)

chiYoungestDF_Men = pd.DataFrame.from_dict(youngestFrequencies_Men, orient='index', columns=['6 - 8'])
chiMiddleDF_Men = pd.DataFrame.from_dict(middleFrequencies_Men, orient='index', columns=['9 - 12 '])
chiHighestDF_Men = pd.DataFrame.from_dict(highestFrequencies_Men, orient='index', columns=['13 - 22'])

chiSquared_DF_Men = chiMiddleDF_Men.combine_first(chiHighestDF_Men.combine_first(chiYoungestDF_Men))
chiSquared_DF_Men = chiSquared_DF_Men.fillna(0)
print(chiSquared_DF_Men)

        13 - 22  6 - 8  9 - 12 
-13.99      1.0    0.0      0.0
-13.90      0.0    0.0      1.0
-13.30      0.0    1.0      0.0
-13.09      1.0    0.0      0.0
-13.08      1.0    0.0      0.0
...         ...    ...      ...
-8.45       0.0    1.0      0.0
-8.25       0.0    1.0      0.0
-8.17       1.0    0.0      0.0
-7.93       1.0    0.0      0.0
-7.63       0.0    0.0      1.0

[279 rows x 3 columns]


In [12]:
#Fill in missing values for men and women so that degrees of freedom and expected values are correctly calculated
index_values_Women = np.arange(-14.5, -6.5, 0.5).round(1)
index_values_Men = np.arange(-14.42, -6.96, 0.01).round(2)
# Filter out the existing index values
new_index_values_women = [x for x in index_values_Women if x not in chiSquared_DF_Women.index]
new_index_values_men = [x for x in index_values_Men if x not in chiSquared_DF_Men.index]

# Create new rows with missing index values and fill with zeros
new_rows_women = pd.DataFrame(0, index=new_index_values_women, columns=chiSquared_DF_Women.columns)
new_rows_men = pd.DataFrame(0, index=new_index_values_men, columns=chiSquared_DF_Men.columns)

chiSquared_DF_Women = pd.concat([chiSquared_DF_Women, new_rows_women])
chiSquared_DF_Men = pd.concat([chiSquared_DF_Men, new_rows_men])

chiSquared_DF_Women.sort_index(inplace=True)
chiSquared_DF_Men.sort_index(inplace=True)


In [13]:
#Building the lists for chi squared excepted values and actual values
expectedValues_chi_Women = []
observedValues_chi_Women = []

observedValues_chi_Men = []
expectedValues_chi_Men = []

#sum totals to use as constants to calc expected values (both values are constant but just for consitencies sake they are treated seperately)
WomenDFtotal = chiSquared_DF_Women.sum().sum()
MenDFtotal = chiSquared_DF_Men.sum().sum()

#degrees of freedom for the chi test (calculated as [num rows - 1][num col - 1]) (both values are constant but just for consitencies sake they are treated seperately)
chiDDOF_Women = (len(chiSquared_DF_Women) - 1)*(len(chiSquared_DF_Women.columns) - 1) #same for both
chiDDOF_Men = (len(chiSquared_DF_Men) - 1)*(len(chiSquared_DF_Men.columns) - 1)

for column in chiSquared_DF_Women:  #expected and observed values for women in age v offset
    for aperOffset_index, row in chiSquared_DF_Women.iterrows():  #df is indexed by the offset so get offset and column for to get observed values
        if row.sum() != 0: #omit cases of row tot equal zero causing f_exp to be zero (works because ddof is constant)
            observedValues_chi_Women.append(chiSquared_DF_Women.loc[aperOffset_index, column])
            expectedValues_chi_Women.append(row.sum() * chiSquared_DF_Women[column].sum()/WomenDFtotal) #expected value formula is row total * column total / total

            
for column in chiSquared_DF_Men:  #expected and observed values for Men in age v offset
    for aperOffset_index, row in chiSquared_DF_Men.iterrows():  #df is indexed by the offset so get offset and column for to get observed values
        if row.sum() != 0: #omit cases of row tot equal zero causing f_exp to be zero (works because ddof is constant)
            observedValues_chi_Men.append(chiSquared_DF_Men.loc[aperOffset_index, column])
            expectedValues_chi_Men.append((row.sum() * chiSquared_DF_Men[column].sum())/MenDFtotal) #expected value formula is row total * column total / total

In [14]:
# Perform chi-squared test on chiSquared_DF_Women 
chi2_stat_Women, chi2_pValue_Women = scipy.stats.chisquare(f_obs= observedValues_chi_Women, f_exp=expectedValues_chi_Women, ddof=chiDDOF_Women)

# Perform chi-squared test on chiSquared_DF_Men
chi2_stat_Men, chi2_pValue_Men = scipy.stats.chisquare(f_obs= observedValues_chi_Men, f_exp=expectedValues_chi_Men, ddof= chiDDOF_Men)

print(str(chi2_stat_Women) + "|" + str(chi2_pValue_Women) + "\n\n" + str(chi2_stat_Men) + "|" + str(chi2_pValue_Men))

846.9660236851139|6.073247654431732e-56

712.7748947008497|nan


In [15]:
#perform the pearson r test (treats age as continuous)

correlationCoef_Women, pearsonR_p_Women = scipy.stats.pearsonr(DependentVariable_Women, IndependentVariable_Women)

correlationCoef_Men, pearsonR_p_Men = scipy.stats.pearsonr(DependentVariable_Men, IndependentVariable_Men)

print(str(correlationCoef_Women) + "|" + str(pearsonR_p_Women) + "\n\n" + str(correlationCoef_Men) + "|" + str(pearsonR_p_Men))

-0.3595585117799993|1.0998505937476708e-58

-0.3752138928653138|5.601559899917795e-36
